In [ ]:
import gzip
from booking_mtlib.normalize import normalize
from booking_mtlib.parallelize import multiprocess
from booking_mtlib.sentence_tokenizer import SentenceTokenizer
import codecs

In [ ]:
tok = ReTokenizer()
words = set([tok.tokenize(l.strip('\n').decode('utf-8').lower())[0] for l in open('/usr/share/dict/words')])
len(words)
import itertools
import random
verbs = set(itertools.chain.from_iterable((tok.tokenize(w)[0] for w in l.strip('\n').decode('utf-8').lower().split(',') if w) for l in open('verb.txt')))
len(verbs)
all_words = verbs | words

In [ ]:
%%time
wg_f = open('word_splits.txt', 'w')

for i, w in enumerate(all_words):
    valid_splits = set()
    splits = [(w[:i], w[i:]) for i in range(1, len(w))]
    for splt in splits:
        if splt[0] in all_words and splt[1] in all_words:
            valid_splits.add(u" ".join(splt))
    if valid_splits:
        wg_f.write("\t".join((w, u",".join(valid_splits))).encode('utf-8'))
        wg_f.write("\n")
wg_f.close()

In [ ]:
all_possible_replaces = defaultdict(set)
f = open('word_graph.txt')
for l in f:
    w, s = l.strip('\n').decode('utf-8').split(u'\t')
    for ws in s.split(u','):
        if ws != w:
            all_possible_replaces[w].add(ws)

f = open('word_splits.txt')
for l in f:
    w, s = l.strip('\n').decode('utf-8').split(u'\t')
    for ws in s.split(u','):
        if ws != w:
            all_possible_replaces[w].add(ws)

f = open('verb.txt')
for l in f:
    ws = set(tok.tokenize(w)[0] for w in l.strip('\n').decode('utf-8').lower().split(',') if w)
    for w1 in ws:
         for w2 in ws:
            if w1 != w2:
                all_possible_replaces[w1].add(w2)